# Assignment 3
write your names here

In [7]:
import numpy as np

In [8]:
def linear_pca(X, k):
    mu=np.mean(X,axis=1)
    X_c=X-np.expand_dims(mu,axis=1)
    [U,Sigma,VT]=np.linalg.svd(X_c,full_matrices=False)
    S=np.expand_dims(Sigma[:k],axis=1)*VT[:k,:]
    return S

In [9]:
# Insert Code for Subtask 1.1 here

In [10]:
def gram_pca(K,k):
    # Insert Code for Subtask 1.2 here
    return S

In [11]:
# Insert Code for Subtask 1.3 here

In [1]:
def compute_gaussian_gram_matrix(X, sigma):
    #Insert Code for Subtask 1.4 here
    return K

def gaussian_kernel_pca(X, k, sigma):
    K=compute_gaussian_gram_matrix(X,sigma)
    S=gram_pca(K,k)
    return S

In [14]:
#Insert Code for Subtask 1.5 here